In [1]:
#import
from sixgill.pipesim import Model
from sixgill.definitions import *
from tkinter import filedialog as fd
from tkinter import *
import pandas as pd

# путь к файлу pips
#filename = r"C:\Users\leone\OneDrive\_Магистратура\3 Семестр\ГПН\models\Python toolkit\Well ESP gas degradation — копия\5303_new.pips"

# диалоговое окно выбора файла pipesim
root = Tk()
root.title("Select PIPS model")
#Tk().mainloop()
filename= fd.askopenfilename() 
root.destroy()

#Connecting to the model
model = Model.open(filename,Units.METRIC)
print("model open")

#get Well and ESP name from model
Well = model.find(component=ModelComponents.WELL)[0]
ESP = model.find(component=ModelComponents.ESP)[0]
pump_name = model.find_components(component=ModelComponents.ESP, Well=Well)[0]['Name']

#



INFO:manta.server.manager:Starting PIPESIM server on thread ID: 3908
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:manta.server.manager:Waiting for PIPESIM server to start.


model open


In [2]:
#test model
model.get_value(ESP,parameter = Parameters.ESP.OPERATINGFREQUENCY)

51.0

In [3]:
# функция рассчета коэффициента по напору  
def calc_gas_correction(gas_fraction_in, Q_morm):
    B = 0
    if gas_fraction_in > 0 and gas_fraction_in < 1:
        B = gas_fraction_in

    calc_gas_correction = {
        0:1,
        1:-9 * B ** 2 + 0.6 * B + 1,                # SPE 117414  стандартный ЭЦН с пределом по газу 25%
        2:-2 * B ** 2 + 0.05 * B + 1,               # SPE 117414 (исправлению по сравнению со статьей верить)  ЭЦН с газостабилизирующим модулем с пределом по газу 50%
        3:-1.4 * B ** 2 + 0.15 * B + 1,             # SPE 117414 ЭЦН с осевым предвключенным модулем с пределом по газу 75%
        4:-4 * B ** 2 + 0.2 * B + 1,                # SPE 117414 (исправлению по сравнению со статьей верить)   ЭЦН с газостабилизирующим модулем с пределом по газу 35%
        5:1 + 0.4461 * B - 5.2506 * B ** 2 *Q_morm,  # собственный анализ ЭЦН + предвключенный фазопреобразователь
        6:1 - 1.8916 * B ** 2 + 9.2057 * B * Q_morm - 22.6222* B ** 2 * Q_morm - 10.7115 * B * Q_morm ** 2      # собственный анализ ЭЦН без модулей
    }
    if calc_gas_correction.get(gas_correct_model) >1:
        return 1
    else:return calc_gas_correction.get(gas_correct_model)
    


In [4]:
# результаты, которые должен вывести pipesim
RESULT_VARIABLES = [SystemVariables.VOLUME_FLOWRATE_OIL_STOCKTANK,
                    SystemVariables.VOLUME_FLOWRATE_WATER_STOCKTANK,
                    SystemVariables.BOTTOM_HOLE_PRESSURE,
                    SystemVariables.ESP_INTAKE_GAS_VOLUME_FRACTION,
                    SystemVariables.ESP_HEAD,
                    SystemVariables.ESP_HEAD_FACTOR,
                    SystemVariables.ESP_INTAKE_TOTAL_VOLUMETRIC_FLOWRATE,
                    SystemVariables.VOLUME_FLOWRATE_LIQUID_STOCKTANK,
                    SystemVariables.OUTLET_WATER_CUT_STOCKTANK,
                    SystemVariables.ESP_INTAKE_PRESSURE
                    ]

gas_correct_model = 2 # модель деградации 

#начальные значения критериев, используемых в расчете
gas_correction = 1
delta_gas_correction = 1

while delta_gas_correction >= 0.001:
    
    esp_motor_params = {
        ESP: {
            Parameters.ESP.HEADFACTOR:gas_correction
        }
    }
    
    model.set_values(esp_motor_params)
        
    
    # Run the model
    print("Running the simulation with Res h_degr={}".format(gas_correction))
    results = model.tasks.ptprofilesimulation.run(Well,profile_variables=[], system_variables=RESULT_VARIABLES)
    
    case = results.cases[0]
    print(results.cases[0])
    
    #максимальный расход по характеристике (Нужно добавить экстраполяцию до H=0)
    esp_results = results.esp_curves[results.cases[0]]
    Q_max = max(esp_results[pump_name][EspCurvesVariables.VARIABLEPERFORMANCECURVE][EspCurvesVariables.FLOWRATE][EspCurvesVariables.VALUES])
    
    # газосодержание и объемный расход на входе в ЭЦН
    B = results.system[SystemVariables.ESP_INTAKE_GAS_VOLUME_FRACTION][case] / 100
    Q = results.system[SystemVariables.ESP_INTAKE_TOTAL_VOLUMETRIC_FLOWRATE][case]
    # отношение подачи ЭЦН к подаче при нулевом напоре
    Q_norm = Q / (Q_max*1000) # ESP_INTAKE_TOTAL_VOLUMETRIC_FLOWRATE выводится в размерности l/d, а  max Q ЭЦН в m3/d (for Units.METRIC)

    new_gas_correction = calc_gas_correction(B, Q_norm)
    delta_gas_correction = abs(new_gas_correction - gas_correction)
    
    if new_gas_correction >= 0.11:
        gas_correction = (new_gas_correction + gas_correction * 2) / 3
    else:
        gas_correction = (0.11 + gas_correction * 2) / 3
    print("ESP_INTAKE_GAS_VOLUME_FRACTION = {}".format(B))
    print("new_gas_correction ={}".format(new_gas_correction))

#Вывод результатов расчета
print("Решение найдено")
df = pd.DataFrame.from_dict(results.node[case], orient='index')
ds = pd.DataFrame.from_dict(results.system, orient='index')
print(ds.iloc[::])
print(df)


Running the simulation with Res h_degr=1


INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is finished.
INFO:sixgill.core.run_or_start_simulation:Simulation run successfully.


Flowrate=480.5917 sbbl/day
ESP_INTAKE_GAS_VOLUME_FRACTION = 0.12099929664554555
new_gas_correction =0.9767683052548438


INFO:sixgill.core.run_or_start_simulation:Simulation is running.


Running the simulation with Res h_degr=0.9922561017516146


...

Flowrate=473.0054 sbbl/day
ESP_INTAKE_GAS_VOLUME_FRACTION = 0.11682126500650758
new_gas_correction =0.9785466473348841


INFO:sixgill.core.run_or_start_simulation:Simulation is running.


Running the simulation with Res h_degr=0.9795838339547348


INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simulation is finished.
INFO:sixgill.core.run_or_start_simulation:Simulation run successfully.


Flowrate=472.8044 sbbl/day
ESP_INTAKE_GAS_VOLUME_FRACTION = 0.11671445576427286
new_gas_correction =0.9785911944195128
Решение найдено
                                  Unit  Flowrate=472.8044 sbbl/day
ESPIntakePressure                 bara                  107.961677
ESPHeadFactor                                             1.000000
OutletWaterCutStockTank              %                   83.800000
ESPIntakeTotalVolumetricFlowrate   L/d                67465.754898
BottomHolePressure                bara                  129.973170
ESPHead                              m                 2670.442468
ESPIntakeGasVolumeFraction           %                   11.671446
                                Unit        Spr        Cpl        ESP
VolumeFlowrateWaterStockTank   sm3/d  62.992370  62.992370  62.992370
VolumeFlowrateOilStockTank     sm3/d  12.177523  12.177523  12.177523
VolumeFlowrateLiquidStockTank  sm3/d  75.169892  75.169892  75.169892
